In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv('GFM_data.csv', sep = '\t')

In [2]:
import tensorflow as tf
import string
import requests

In [3]:
text = df.get('Text').to_numpy()

In [20]:
import string
"""#redo
def extract_entities(text):
    names = []
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label'):
                names.append(' '.join(c[0] for c in chunk.leaves()))
    new_text = text
    for name in names:
        if name in text:
            new_text = new_text.replace(name, 'NLP')
    return new_text

def clean_text(x):
    ## removing names
    x = extract_entities(x)
    ## normalizing text by stripping white space and lower casing
    x =  x.lower().strip()
    ## removing urls
    x = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', x)
    ## removing phone numbers
    x = re.sub('\([0-9]{3}\)\s*[0-9]{3}-[0-9]{4}','',x)
    ## strip all non alphanumeric things
    x = re.sub('\n',' ',x)
    x = re.sub("[^a-zA-Z0-9 #]",'',x)
    x = re.sub("\s+",' ',x)
    text = x.replace('\n', ' ').lower()# lowercase text
    text = REPLACE_IP_ADDRESS.sub('', text) # remove ip address
    text = REPLACE_BY_SPACE_RE.sub(' ',text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('',text)# delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join([w for w in text.split() if not w in STOPWORDS])# delete stopwords from text
    
    return text
"""
def clean_data(txt_elmt):
    # txt_elmt is each element (each post) in the "text" array
    tokens = txt_elmt.split() # each word
    
    # nothing changes. except removing punctuation
    mytable = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(mytable) for word in tokens]
    return [word.lower() for word in tokens if word.isalpha()]

#for i in text:
# try with text[0] first
tokens = clean_data(text[0]) # list of tokens
length = 51 # learn initial 50 words, predict the next word
lines = [] # lst of all training sequences
sufficiency = 200_000
"""
for i in range(len(tokens)):
    sequence = tokens[i: i + length] # every 50-word
    lines += [' '. join(sequence)]
    if i > sufficiency:
        break
"""
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i > 200000:
        break

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import LambdaCallback

In [6]:
tokenizer = Tokenizer(oov_token="<OOV>") # OOV adds new words outside the trained words
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines) # all words become numbers

In [7]:
"""
X = []
y = []
maxlen=50
for i in range(len(sequences)):
    X.append(sequences[i][:-1])
    y.append(sequences[-1])
"""
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

In [8]:
# one-hot encoding
vocab_size = len(tokenizer.word_index) + 1
y = to_categorical(y, num_classes=vocab_size)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [9]:
y[0].size

121

In [10]:
sequence_length = len(X[0])
 
# LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = sequence_length)) # input dim - vocabsize, output dim = 50
# 100 hidden layers
model.add(LSTM(100, return_sequences=True)) # do twice, return_sequences=True
model.add(LSTM(100)) 
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            6050      
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 121)               12221     
Total params: 169,171
Trainable params: 169,171
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.fit(X, y, batch_size=256, epochs=100)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 4.7956 - accuracy: 0.0088
Epoch 2/100
1/1 [==============================] - 0s 107ms/step - loss: 4.7937 - accuracy: 0.0175
Epoch 3/100
1/1 [==============================] - 0s 94ms/step - loss: 4.7916 - accuracy: 0.0175
Epoch 4/100
1/1 [==============================] - 0s 96ms/step - loss: 4.7890 - accuracy: 0.0175
Epoch 5/100
1/1 [==============================] - 0s 98ms/step - loss: 4.7857 - accuracy: 0.0263
Epoch 6/100
1/1 [==============================] - 0s 97ms/step - loss: 4.7813 - accuracy: 0.0526
Epoch 7/100
1/1 [==============================] - 0s 100ms/step - loss: 4.7754 - accuracy: 0.0614
Epoch 8/100
1/1 [==============================] - 0s 97ms/step - loss: 4.7673 - accuracy: 0.0614
Epoch 9/100
1/1 [==============================] - 0s 98ms/step - loss: 4.7553 - accuracy: 0.0614
Epoch 10/100
1/1 [==============================] - 0s 98ms/step - loss: 4.7368 - accuracy: 0.0614
Epoch 11/100
1/1 [=

1/1 [==============================] - 0s 147ms/step - loss: 2.1064 - accuracy: 0.2719
Epoch 84/100
1/1 [==============================] - 0s 168ms/step - loss: 2.0550 - accuracy: 0.3509
Epoch 85/100
1/1 [==============================] - 0s 147ms/step - loss: 2.0424 - accuracy: 0.3596
Epoch 86/100
1/1 [==============================] - 0s 133ms/step - loss: 2.0349 - accuracy: 0.3509
Epoch 87/100
1/1 [==============================] - 0s 131ms/step - loss: 2.0717 - accuracy: 0.3421
Epoch 88/100
1/1 [==============================] - 0s 127ms/step - loss: 2.0604 - accuracy: 0.2544
Epoch 89/100
1/1 [==============================] - 0s 143ms/step - loss: 1.9759 - accuracy: 0.3684
Epoch 90/100
1/1 [==============================] - 0s 149ms/step - loss: 1.9905 - accuracy: 0.3421
Epoch 91/100
1/1 [==============================] - 0s 145ms/step - loss: 1.9907 - accuracy: 0.3772
Epoch 92/100
1/1 [==============================] - 0s 131ms/step - loss: 1.9461 - accuracy: 0.3596
Epoch 93/100


In [17]:
seed_text=lines[len(lines) - 1]
seq_length = X.shape[1]
seed_text

'six children some of whom witnessed this horrific act of violence anyone wishing to send cards letters of encouragement andor contributions in the form of a money order or check may do so by mail at julia y jackson co ben crump law pllc s calhoun street tallahassee fl attn adner'

In [18]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []

    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
        if index == y_predict:
            predicted_word = word
            break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
    return ' '.join(text)

In [19]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

'adner'

https://kgptalkie.com/text-generation-using-tensorflow-keras-and-lstm/

https://www.youtube.com/watch?v=VAMKuRAh2nc&t=1607s&ab_channel=KGPTalkie